In [23]:
from dataclasses import dataclass
from typing import List, Union, Optional, Dict

import copy
import sys
ahc008_path = "/Users/takizawa/dev/ahc/ahc008"
if ahc008_path not in sys.path:
    sys.path.append(ahc008_path)
print(sys.path)

from main import Floor, Point, Tile, move_char_to_diff

['/Users/takizawa/dev/ahc/ahc008/notebook', '/Users/takizawa/.pyenv/versions/3.8.2/lib/python38.zip', '/Users/takizawa/.pyenv/versions/3.8.2/lib/python3.8', '/Users/takizawa/.pyenv/versions/3.8.2/lib/python3.8/lib-dynload', '', '/Users/takizawa/dev/ahc/ahc008/.venv/lib/python3.8/site-packages', '/Users/takizawa/dev/ahc/ahc008']


In [2]:
# @dataclass
# class Steps:
#     u: int = 0
#     d: int = 0
#     l: int = 0
#     r: int = 0
    
#     def __repr__(self):
#         return f"({self.u}, {self.d}, {self.l}, {self.r})"

In [3]:
class Steps(dict):
    def __init__(self, U=0, D=0, L=0, R=0, *arg,**kw):
        super(Steps, self).__init__(*arg, **kw)
        self["U"] = U
        self["D"] = D
        self["L"] = L
        self["R"] = R
        
    def __repr__(self):
        return f"({self['U']}, {self['D']}, {self['L']}, {self['R']})"

In [4]:
steps = Steps(1,0,0,4)
steps

(1, 0, 0, 4)

In [6]:
iter(steps)

In [7]:
steps["U"]

1

In [8]:
for k, v in steps.items():
    print(k,v)

U 1
D 0
L 0
R 4


In [10]:
steps

(1, 0, 0, 4)

In [11]:
steps["U"] += 1
steps

(2, 0, 0, 4)

In [12]:
floor = Floor(5, 1)

In [13]:
floor_1 = Floor(5, 2)
floor_1.update_tile(Point(2,5), Tile.PARTITION)
floor_1.update_tile(Point(3,5), Tile.PARTITION)
floor_1.update_tile(Point(4,5), Tile.PARTITION)
# floor_1.update_tile(Point(3,6), Tile.PARTITION)

In [14]:
floor_1

111111111
111111111
110002011
110002011
110002011
110000011
110000011
111111111
111111111

In [27]:
floor.tiles[2]

[<Tile.WALL: 1>,
 <Tile.EMPTY: 0>,
 <Tile.EMPTY: 0>,
 <Tile.EMPTY: 0>,
 <Tile.EMPTY: 0>,
 <Tile.EMPTY: 0>,
 <Tile.WALL: 1>]

In [15]:
START = Point(6,3)
GOAL = Point(2,6)

In [17]:
def cal_steps(start, goal):
    diff = GOAL - START
    steps = Steps()
    # 上下
    if diff.x > 0:
        steps["D"] = diff.x
    else:
        # 正の数にする
        steps["U"] = -diff.x
    
    # 左右
    if diff.y > 0:
        steps["R"] = diff.y
    else:
        steps["L"] = -diff.y
    
    return steps

In [18]:
cal_steps(START,GOAL)

(4, 0, 0, 3)

In [70]:
steps = cal_steps(START,GOAL)

In [71]:
steps

(4, 0, 0, 3)

In [72]:
for step in steps.items():
    print(step)

('U', 4)
('D', 0)
('L', 0)
('R', 3)


In [74]:
move_char_to_diff

{'U': PointDiff(x=-1, y=0),
 'D': PointDiff(x=1, y=0),
 'L': PointDiff(x=0, y=-1),
 'R': PointDiff(x=0, y=1),
 '.': PointDiff(x=0, y=0)}

In [90]:
Point(1,2) == Point(1,2)

True

In [36]:
def solve_route(start, goal, floor)-> List[Point]:
    """startからgoalまでの経路のPointのListを返す
    経路長は最短であることを前提とする。
    経路が見つからなければ空のListを返す。
    
    TODO: 経路長が最短でない場合
    """
    def dfs(start, goal, steps) -> List[Point]:
        if start == goal:
            return [goal]
        
        for step_dir, step_cnt in steps.items():
            if step_cnt == 0:
                continue
            
            diff = move_char_to_diff[step_dir]
            next_point = start + diff
            
            if floor.get_tile(next_point) not in [Tile.WALL, Tile.PARTITION]:
                next_steps = copy.deepcopy(steps)
                next_steps[step_dir] -= 1
                result = dfs(next_point, goal, next_steps)
                if len(result) > 0:
                    return [start] + result
        return []
    
    steps = cal_steps(start, goal)
    return dfs(start, goal, steps)


In [37]:
solve_route(START, GOAL, floor_1)

[Point(x=6, y=3),
 Point(x=5, y=3),
 Point(x=5, y=4),
 Point(x=5, y=5),
 Point(x=5, y=6),
 Point(x=4, y=6),
 Point(x=3, y=6),
 Point(x=2, y=6)]

In [41]:
# 同じ場所
solve_route(Point(3,3),Point(3,3), floor_1)

[Point(x=3, y=3)]

In [40]:
# GOALにPARTITIONがある場合経路はない
solve_route(Point(3,3),Point(2,5), floor_1)

[]